# Numerical semidefinite program solver for the three-box problem

## Context

In this notebook the optimal expected correct-guessing probability $\text{P}_\text{corr}^*$ is numerically computed by using the *picos* package to solve complex semidefinite program (SDP). Specifically, we solve the following SDP:

$$
\begin{align}
    \text{P}_\text{corr}^* = \min_X\quad & \text{Tr}(X) & & \\
    \text{subject to } \quad & X \succcurlyeq \eta_j \rho_j & j&=A,B,C.
\end{align}
$$

## Outputs

- Numerically computed minimum expected error probability

In [8]:
import numpy as np
from scipy.special import comb, binom
from itertools import combinations
import picos

In [35]:
# simulation parameters

# simulation parameters

n = 1 # half-number of photons
N = 2*n # total number of photons
d = 2**N # hilbert space dimension
etaA = 1/3. # prior probability of box A
etaB = 1/3. # prior probability of box B
etaC = 1-etaA-etaB # prior probability of box C

In [36]:
# load full density matrices

rhoA = np.load("../full_density_matrices/rhoA_N{:d}_num.npy".format(N))
rhoB = np.load("../full_density_matrices/rhoB_N{:d}_num.npy".format(N))
rhoC = np.load("../full_density_matrices/rhoC_N{:d}_num.npy".format(N))

# Semidefinite Programming Solver

In [37]:
# set picos constants

rA = picos.Constant(etaA*rhoA)
rB = picos.Constant(etaB*rhoB)
rC = picos.Constant(etaC*rhoC)

In [38]:
# sdp statement

P = picos.Problem()
X = picos.HermitianVariable("X", rA.shape)
P.set_objective("min", picos.trace(X))
P.add_constraint( X - rA >> 0)
P.add_constraint( X - rB >> 0)
P.add_constraint( X - rC >> 0)

print(P)

Complex Semidefinite Program
  minimize tr(X)
  over
    4×4 hermitian variable X
  subject to
    X - [4×4] ≽ 0
    X - [4×4] ≽ 0
    X - [4×4] ≽ 0


In [39]:
# solve

P.solve(solver = "cvxopt")
Perr = 1-P.value

In [34]:
# print and save data

print("\nOptimal expected error probability:", Perr)
#print("Optimal X:", X.value, sep="\n")

np.savetxt("data/perr_md_2bp_num_N{:d}.txt".format(N), [Perr])


Optimal expected error probability: 0.2767895504622233
